In [2]:
import numpy as np

In [3]:
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import Normalize
import matplotlib.lines as mlines
from scipy.interpolate import UnivariateSpline
from tqdm import tqdm

In [4]:
from MCEq.version import __version__

print(__version__)

1.2.6


In [5]:
from MCEq.core import MCEqRun
import mceq_config as config
import crflux.models as pm

In [6]:
#config.debug_level = 2
#config.enable_default_tracking = False
#config.mceq_db_fname = "mceq_db_lext_dpm193_v150_sstar.h5"
#config.adv_set["disable_charm_pprod"] = True

In [7]:
# Average mass number may be wrong but not relevant
mceq_air = MCEqRun(
    interaction_model="SIBYLL23C",
    theta_deg=0.0,
    primary_model=(pm.GlobalSplineFitBeta, None),
    density_model = ('MSIS00_IC',('South Pole','January')),
)

MCEqRun::set_interaction_model(): SIBYLL23C
ParticleManager::_init_default_tracking(): Initializing default tracking categories (pi, K, mu)
MCEqRun::set_density_model(): Setting density profile to MSIS00_IC ('South Pole', 'January')
MSIS00IceCubeCentered::set_theta(): latitude = -90.00 for zenith angle =  0.00
MCEqRun::set_primary_model(): GlobalSplineFitBeta 


In [8]:
mceq_air.set_interaction_model("SIBYLL23C", force=True)
# inject_scenario(pC_scen, mceq_air)
#mceq_air.set_primary_model(
#    pm.GlobalSplineFitBeta(None),
#)  # pm.GaisserStanevTilav('4-gen'))#
mceq_air.set_theta_deg(0)

MCEqRun::set_interaction_model(): SIBYLL23C
MCEqRun::set_primary_model(): GlobalSplineFitBeta 


In [9]:
cos_thetas = np.arange(0.8, 1.001, 0.1)
thetas = np.degrees(np.arccos(cos_thetas))
print(cos_thetas, thetas)

[0.8 0.9 1. ] [36.86989765 25.84193276  0.        ]


In [12]:
seasons = ['January', 'April', 'July']
surface_flux_GSF = []

for season in tqdm(seasons):  # Loop over seasons
    season_flux = []  # Temporary container for the current season's results
    for ia, theta in enumerate(thetas):  # Loop over angles
        mceq_air.set_theta_deg(theta)
        
        # Optionally set the season-dependent atmospheric model here
        mceq_air.density_model.set_season(season)  # Uncomment and adjust if applicable
        
        # Solve for the current angle and season
        mceq_air.solve()
        
        # Append the solution for the current angle
        season_flux.append(
            mceq_air.get_solution("mu+", mag=0) + mceq_air.get_solution("mu-", mag=0)
        )
    # Append the results for the current season
    surface_flux_GSF.append(season_flux)

  0%|          | 0/3 [00:00<?, ?it/s]

MSIS00IceCubeCentered::set_theta(): latitude = -90.00 for zenith angle = 25.84
MSIS00IceCubeCentered::set_theta(): latitude = -90.00 for zenith angle =  0.00


 33%|███▎      | 1/3 [00:08<00:17,  8.66s/it]

MSIS00IceCubeCentered::set_theta(): latitude = -90.00 for zenith angle = 36.87
MSIS00IceCubeCentered::set_theta(): latitude = -90.00 for zenith angle = 25.84
MSIS00IceCubeCentered::set_theta(): latitude = -90.00 for zenith angle =  0.00


 67%|██████▋   | 2/3 [00:17<00:08,  8.69s/it]

MSIS00IceCubeCentered::set_theta(): latitude = -90.00 for zenith angle = 36.87
MSIS00IceCubeCentered::set_theta(): latitude = -90.00 for zenith angle = 25.84
MSIS00IceCubeCentered::set_theta(): latitude = -90.00 for zenith angle =  0.00


100%|██████████| 3/3 [00:26<00:00,  8.70s/it]


In [13]:
np.shape(surface_flux_GSF)

(3, 3, 121)

In [14]:
config.debug_level = 0

In [16]:
cr_grid = mceq_air.e_grid[30:-10] # cut because higher energies are not relevant for analysis

In [18]:
ground_muspec_prim_energies = []

for season in tqdm(seasons):  # Loop over seasons
    season_energies = []  # Temporary container for the current season's results
    for ei, eprim in enumerate(tqdm(cr_grid)):  # Loop over primary energies
        theta_energies = np.zeros((thetas.shape[0], mceq_air.dim))
        mceq_air.set_single_primary_particle(E=eprim, pdg_id=2212)
        for ia, theta in enumerate(thetas):  # Loop over angles
            mceq_air.set_theta_deg(theta)
            
            # Optionally set the season-dependent atmospheric model here
            mceq_air.density_model.set_season(season)  # Uncomment and adjust if applicable
            
            mceq_air.solve()
            theta_energies[ia, :] = mceq_air.get_solution("mu+", mag=0) + mceq_air.get_solution("mu-", mag=0)
        
        season_energies.append(theta_energies)
    # Append the results for the current season
    ground_muspec_prim_energies.append(season_energies)

100%|██████████| 3/3 [35:17<00:00, 705.97s/it]


In [19]:
import pickle

pickle.dump(
    [mceq_air.e_grid, cos_thetas, cr_grid, ground_muspec_prim_energies[0], ground_muspec_prim_energies[1], ground_muspec_prim_energies[2]],
    open("ground_muspec_prim_energies_season.pkl", "wb"),
)
pickle.dump(
    [mceq_air.e_grid, cos_thetas, surface_flux_GSF[0], surface_flux_GSF[1] , surface_flux_GSF[2]],
    open("surface_fluxes_season.pkl", "wb"),
)